# Грузим библиотеки

In [ ]:


from flask import Flask
from flask import request
import traceback


import family_budget


application = Flask(__name__)  # Change assignment here



@application.route("/")  # Change your route statements
def hello():
    return "Hello World!"





@application.route('/family_budget', methods=['GET', 'POST'])
def fb():
    try:
        #json_params = json.loads(request.get_data())
        
        family_budget.main()
        
        return 'END Tips'
    except:
        traceback.print_exc()
        return 'ERROR Tips'



if __name__ == "__main__":
    application.run()


In [ ]:
import requests
import access
import json
from datetime import datetime
from time import sleep

token = access.token()
api = access.api()
d_google_maps_api = access.google_maps_api_key()

def get_updates():
    url = api + token + '/getUpdates'
    r = requests.get(url)
    return r.json()



r = get_updates()


In [3]:
import os
from telegram.ext import Updater

TOKEN = "382244799:AAFfN3evzGDQaRevpW5xqZ1AEovvdRCWk-0"
PORT = int(os.environ.get('PORT', '5000'))
updater = Updater(TOKEN)
# add handlers
updater.start_webhook(listen="0.0.0.0",
                      port=PORT,
                      url_path=TOKEN)
updater.bot.set_webhook("https://family-budget-.herokuapp.com/382244799:AAFfN3evzGDQaRevpW5xqZ1AEovvdRCWk-0")
updater.idle()

BadRequest: Bad webhook: getaddrinfo: no address associated with hostname